## Theory

An **autoencoder** is a type of **neural network** used mainly for **unsupervised learning**. It is designed to **encode** input data  and then **reconstruct** the original input from this compressed representation.

#### How It Works:

An autoencoder consists of two main parts:

- **Encoder:** Compresses the input data into a lower-dimensional representation (latent space).
- **Decoder:** Reconstructs the original input from this compressed representation.


## Implementation

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
# restart the kernel and it works

In [3]:
transform = transforms.ToTensor()

mnist_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

data_loader = torch.utils.data.DataLoader(dataset=mnist_data, batch_size=64, shuffle=True)

100%|██████████| 9.91M/9.91M [00:09<00:00, 1.05MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 103kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 851kB/s] 
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.66MB/s]


In [ ]:
dataiter = iter(data_loader)
images, labels = next(dataiter)
print(torch.min(images), torch.max(images))
# As seen, the tensor values ranges from 0 and 1

tensor(0.) tensor(1.)


In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        # In the beginneing image size: N, 784

        # Drastically reduce input image size
        self.encoder = nn.Sequential(
            nn.Linear(28*28, 128), # Linear Layer, reduce size from N, 784 -> N, 128
            nn.ReLU(), # Activation function
            nn.Linear(128, 64), # Linear Layer, reduce size from N, 128 -> N, 64
            nn.ReLU(),
            nn.Linear(64, 12), # Linear Layer, reduce size from N, 64 -> N, 12
            nn.ReLU(),
            nn.Linear(12, 3), # Linear Layer, reduce size from N, 12 -> N, 3
        )

        # Increase the output size to the original image size (oppostie of before)
        self.decoder = nn.Sequential(
            nn.Linear(3, 12), 
            nn.ReLU(),
            nn.Linear(12, 64), 
            nn.ReLU(),
            nn.Linear(64, 128), 
            nn.ReLU(),
            nn.Linear(128, 28*28), 
            nn.Sigmoid() # Activation function to get output between 0 and 1
        )


    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded